In [ ]:
from src.Evaluation.Evaluator import EvaluatorHoldout
import scipy.sparse as sps
import random
import os
import numpy as np

import pandas as pd
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
dataframe_items=pd.read_csv("Dataset/data_ICM_metadata.csv",low_memory=True)
ICM=sps.csr_matrix((dataframe_items["data"].values,(dataframe_items["item_id"].values.astype(int),dataframe_items["feature_id"].values)))

URM_train = sps.load_npz('Dataset/URM_train.npz')
URM_test_complete = sps.load_npz('Dataset/URM_test.npz')

URM_all=URM_train + URM_test_complete

In [ ]:
from src.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])



def objective_function_slim(trial):
    alpha_stacked=trial.suggest_float('alpha_stacked',0.000000001,0.9999)
    params = {
        'l1_ratio': trial.suggest_float('l1_ratio', 1e-5, 1, log=True),
        'alpha': trial.suggest_float('alpha', 1e-5, 100, log=True),
        'topK': trial.suggest_int('topK', 5, 2500, log=True),
        'positive_only': trial.suggest_categorical("positive_only", [True, False]),
    }
    URM_stacked=sps.vstack([URM_train*alpha_stacked,(ICM.T)*(1-alpha_stacked)])
    recommender_instance = SLIMElasticNetRecommender(URM_stacked)
    recommender_instance.fit(**params)

    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    return result_df.loc[10, "MAP"]

In [ ]:
from optuna.samplers import RandomSampler
import optuna
database_path='hypertuning_results/tuning_slim_alpha_stacked.db'
study=optuna.load_study(
    study_name='hyperparameters_tuning_slim',
    storage=f'sqlite:///{database_path}'
)
study.sampler=RandomSampler(seed=SEED)




study.optimize(objective_function_slim, n_trials=150, n_jobs=6)